In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

### Import

In [8]:
import numpy as np
import plotly
import pandas as pd
import plotly.graph_objects as go
# import geopandas as gpd
from scipy.stats.stats import pearsonr
pd.set_option('display.max_columns', None)  

# Load *DF*

In [9]:
df = pd.read_csv(r"Unique stops - last attempt 2.csv")

# Create Data

## Create *Coordinates Dict*

In [10]:
"""
coordinates_dict holds all citations, by latitude/longitude.
each long/lat entry of coordinates_dict holds a small list, which counts the number of observations for each race 
at that location.
coordinates_dict[longitude][latitude](race) = 
        # of observations of this race at this location[0,5], [sum(0-5)==all stops at this location]

each internal list is organized as follows:
[asian, black, hispanic, native american, other, white, TOTAL STOPS, %asian, %black, %hispanic, %native american, %other, %white]
"""

#SHOULD THIS BE CHANGED FROM .005 TO .01 AS IS DONE BELOW????
coordinates_dict = {}
for x in np.arange(-77.6,-76.5,0.005):
    coordinates_dict[np.around(x,4)] = {}

for x in coordinates_dict:
    for y in np.arange(38.5,39.5,0.005):
        coordinates_dict[x][np.around(y,4)] = {"Total Unique Stops":([0]*6), "Number of Citations":([0]*6), "Search Conducted":([0]*6), "Probable Cause":([0]*6), "Arrested":([0]*6), "Accidents":([0]*6),}

"""
CHANGING ROUNDING SIZE FROM .005 TO .01
"""
def rounder(x):#rounds to nearest .005
    return np.around(.01 * round(x/.01),4)

#for assigning to list in coordinates_dict
race_key_dict = {'ASIAN':0, 'BLACK':1, 'HISPANIC':2, 'NATIVE AMERICAN':3, 'OTHER':4, 'WHITE':5}

#assigning values to coordinate_dict's race-list
for i in range(len(df)):
    try:    
        coordinates_dict[rounder(df["Longitude"].iloc[i])][rounder(df["Latitude"].iloc[i])]["Total Unique Stops"][race_key_dict[df["Race"].iloc[i]]] += 1
        coordinates_dict[rounder(df["Longitude"].iloc[i])][rounder(df["Latitude"].iloc[i])]["Number of Citations"][race_key_dict[df["Race"].iloc[i]]] += df["Num_Citations"].iloc[i]
        if df["D_Arrested"].iloc[i]==1:
            coordinates_dict[rounder(df["Longitude"].iloc[i])][rounder(df["Latitude"].iloc[i])]["Arrested"][race_key_dict[df["Race"].iloc[i]]] += 1
        if df["D_SearchConducted"].iloc[i]==1:
            coordinates_dict[rounder(df["Longitude"].iloc[i])][rounder(df["Latitude"].iloc[i])]["Search Conducted"][race_key_dict[df["Race"].iloc[i]]] += 1
        if df["D_ProbableCause"].iloc[i]==1:
            coordinates_dict[rounder(df["Longitude"].iloc[i])][rounder(df["Latitude"].iloc[i])]["Probable Cause"][race_key_dict[df["Race"].iloc[i]]] += 1
        if df["D_Accident"].iloc[i]==1:
            coordinates_dict[rounder(df["Longitude"].iloc[i])][rounder(df["Latitude"].iloc[i])]["Accidents"][race_key_dict[df["Race"].iloc[i]]] += 1
    except:
        pass

for x in coordinates_dict:#longitude
    for y in coordinates_dict[x]:           #latitude
        for dicts in coordinates_dict[x][y]:
            coordinates_dict[x][y][dicts].append(sum(z for z in coordinates_dict[x][y][dicts]))    #append sum to end        

for x in coordinates_dict:
    for y in coordinates_dict[x]:
        for dicts in coordinates_dict[x][y]:
            if not all(z==0 for z in coordinates_dict[x][y][dicts]):
                for z in coordinates_dict[x][y][dicts][:6]:
    #                 print(z, coordinates_dict[x][y][6], z/coordinates_dict[x][y][6])
                    coordinates_dict[x][y][dicts].append(z/coordinates_dict[x][y][dicts][6])
            else:
                coordinates_dict[x][y][dicts].extend([0]*6)

## Def *divide* function

In [11]:
def divide(num, denom):
    if denom!=0:
        return num/denom
    else:
        return 0

## Create Sub-*DF*s

In [35]:
pct_stopped_df = pd.DataFrame({
    
     'Total Stopped': [coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian as Percent of Stopped': [divide(coordinates_dict[x][y]["Total Unique Stops"][0], coordinates_dict[x][y]["Total Unique Stops"][0]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black as Percent of Stopped': [divide(coordinates_dict[x][y]["Total Unique Stops"][1], coordinates_dict[x][y]["Total Unique Stops"][1]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic as Percent of Stopped': [divide(coordinates_dict[x][y]["Total Unique Stops"][2], coordinates_dict[x][y]["Total Unique Stops"][2]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American as Percent of Stopped': [divide(coordinates_dict[x][y]["Total Unique Stops"][3], coordinates_dict[x][y]["Total Unique Stops"][3]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other as Percent of Stopped': [divide(coordinates_dict[x][y]["Total Unique Stops"][4], coordinates_dict[x][y]["Total Unique Stops"][4]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White as Percent of Stopped': [divide(coordinates_dict[x][y]["Total Unique Stops"][5], coordinates_dict[x][y]["Total Unique Stops"][5]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]       
})

In [15]:
prob_of_search_df = pd.DataFrame({
     'Percent Searched': [divide(coordinates_dict[x][y]["Search Conducted"][6], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Percent Searched': [divide(coordinates_dict[x][y]["Search Conducted"][0], coordinates_dict[x][y]["Total Unique Stops"][0]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Percent Searched': [divide(coordinates_dict[x][y]["Search Conducted"][1], coordinates_dict[x][y]["Total Unique Stops"][1]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Percent Searched': [divide(coordinates_dict[x][y]["Search Conducted"][2], coordinates_dict[x][y]["Total Unique Stops"][2]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Percent Searched': [divide(coordinates_dict[x][y]["Search Conducted"][3], coordinates_dict[x][y]["Total Unique Stops"][3]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Percent Searched': [divide(coordinates_dict[x][y]["Search Conducted"][4], coordinates_dict[x][y]["Total Unique Stops"][4]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Percent Searched': [divide(coordinates_dict[x][y]["Search Conducted"][5], coordinates_dict[x][y]["Total Unique Stops"][5]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]       
    
})

In [28]:
prob_of_citation_df = pd.DataFrame({
     
     'Percent Cited': [divide(coordinates_dict[x][y]["Number of Citations"][6], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Percent Cited': [divide(coordinates_dict[x][y]["Number of Citations"][0], coordinates_dict[x][y]["Total Unique Stops"][0]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Percent Cited': [divide(coordinates_dict[x][y]["Number of Citations"][1], coordinates_dict[x][y]["Total Unique Stops"][1]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Percent Cited': [divide(coordinates_dict[x][y]["Number of Citations"][2], coordinates_dict[x][y]["Total Unique Stops"][2]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Percent Cited': [divide(coordinates_dict[x][y]["Number of Citations"][3], coordinates_dict[x][y]["Total Unique Stops"][3]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Percent Cited': [divide(coordinates_dict[x][y]["Number of Citations"][4], coordinates_dict[x][y]["Total Unique Stops"][4]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Percent Cited': [divide(coordinates_dict[x][y]["Number of Citations"][5], coordinates_dict[x][y]["Total Unique Stops"][5]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]       
    
})

## Def corr matrix function

In [36]:
def create_corr_matrix_dataframe(df1, df2, df1_var, df2_var, drop_top_corner):
    temp = np.zeros((7,7))
    for i, col in enumerate(temp):
        for j, row in enumerate(temp[i]):
            temp[i][j] = round(pearsonr(df1.iloc[:, i], df2.iloc[:, j])[0], 2)


    corr_df = pd.DataFrame(temp)

    corr_df.columns = df1.columns
    
    corr_df.rename({0:f'Total {df2_var}', 1: f'Asian {df2_var}', 2: f'Black {df2_var}', 3: f'Hispanic {df2_var}', 4: f'Native American {df2_var}', 5: f'Other {df2_var}', 6: f'White {df2_var}'}, inplace=True)
    
    if drop_top_corner == True:
        corr_df.iloc[0] = np.nan

    import seaborn as sns
    cm = sns.light_palette("green", as_cmap=True)


    with pd.option_context('display.precision', 2):
        html1 = (corr_df.style
                .background_gradient(cmap=cm, axis=None, high=0.2)
               )

    html1.columns.name = f'Correlation Matrix: {df1_var} vs. {df2_var}'
    return(html1)
#     return(corr_df)

# Create Correlation Matrices

## $Corr(\% stopped, \, \mathbb{P}(Searched) \,)$

In [37]:
create_corr_matrix_dataframe(pct_stopped_df, pct_searched_df, 'stopped', 'searched', False)

Correlation Matrix: stopped vs. searched,Total Stopped,Asian as Percent of Stopped,Black as Percent of Stopped,Hispanic as Percent of Stopped,Native American as Percent of Stopped,Other as Percent of Stopped,White as Percent of Stopped
Total searched,-0.00,0.09,0.04,0.05,0.13,0.05,0.00
Asian searched,-0.08,0.14,0.03,0.05,0.09,0.05,-0.03
Black searched,-0.02,0.10,0.13,0.02,0.06,0.03,-0.01
Hispanic searched,-0.06,0.09,0.00,0.13,0.07,0.05,-0.04
Native American searched,-0.02,0.13,0.07,0.05,0.19,0.08,0.00
Other searched,-0.08,0.12,0.01,0.04,0.09,0.12,-0.02
White searched,-0.07,0.06,-0.03,-0.02,0.04,0.06,0.08


## $Corr(\% stopped, \, \mathbb{P}(Cited) \,)$

In [38]:
create_corr_matrix_dataframe(pct_stopped_df, prob_of_citation_df, 'stopped', 'cited', 0)

Correlation Matrix: stopped vs. cited,Total Stopped,Asian as Percent of Stopped,Black as Percent of Stopped,Hispanic as Percent of Stopped,Native American as Percent of Stopped,Other as Percent of Stopped,White as Percent of Stopped
Total cited,-0.07,0.13,-0.00,0.08,0.29,0.12,-0.02
Asian cited,-0.22,0.51,0.04,0.17,0.25,0.33,-0.09
Black cited,-0.15,0.25,0.31,0.13,0.17,0.23,-0.17
Hispanic cited,-0.19,0.29,-0.01,0.45,0.20,0.29,-0.12
Native American cited,-0.13,0.20,0.00,0.09,0.51,0.17,-0.06
Other cited,-0.24,0.33,0.04,0.14,0.25,0.51,-0.12
White cited,-0.25,0.12,-0.13,-0.01,0.11,0.17,0.24


In [39]:
prob_of_citation_df

,Percent Cited,Asian Percent Cited,Black Percent Cited,Hispanic Percent Cited,Native American Percent Cited,Other Percent Cited,White Percent Cited
0,1.0000,0.0,1.0,0.0,0.0,0.0,1.000000
1,1.0000,0.0,1.0,0.0,0.0,0.0,0.000000
2,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...
1331,6.0000,0.0,0.0,0.0,0.0,0.0,6.000000
1332,6.0000,6.0,0.0,0.0,0.0,0.0,0.000000
1333,0.0625,0.0,0.0,0.0,0.0,0.0,0.142857
1334,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000


In [40]:
df.head()

,Latitude,Longitude,Num_Citations,D_SearchConducted,Race,D_Arrested,D_ProbableCause,D_Accident
0,39.06243,-77.18113,1,0,ASIAN,0,0,0
1,39.19843,-77.25368,0,0,ASIAN,0,0,0
2,39.10619,-77.23292,0,0,ASIAN,0,0,0
3,39.00158,-77.02608,1,0,ASIAN,0,0,0
4,39.15022,-77.06772,0,0,ASIAN,0,0,0
